# Importa pacotes

In [1]:
import llm
from sklearn.metrics import classification_report
from tqdm import tqdm
tqdm.pandas()

# Exemplo básico

In [2]:
llama = llm.get_model("llama3")

In [ ]:
%%time

resposta = llama.prompt(
    """
    Classifique o texto abaixo. Responda apenas com a classe, sem explicações.

    Texto: O Botafogo ganhou do PSG na Copa do Mundo.

    Classes: Política, Esportes, Economia, Outros.
    """
)

print(resposta)

Esportes
CPU times: user 69.1 ms, sys: 1.95 ms, total: 71.1 ms
Wall time: 13.4 s


# Exemplo com dataset real

In [4]:
from datasets import load_dataset

ementas = load_dataset("azmina/ementas_anotadas_inteiroteor")

ementas_test = ementas['test'].to_pandas().sample(10,random_state=222)


/home/abitporu/Documents/abraji/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def classificar_pl(model, texto):
    prompt = f"""
    Classifique se o PL é favorável ou contrário ao direito das mulheres. Responda apenas com 1 ou 0, sem explicações, de acordo com as definições:

    1: significa que o PL é desfavorável ao direito das mulheres.
    0: significa que o PL é favorável ao direito das mulheres.  

    ---

    Texto: {texto}

    Classes: 1, 0
    """
    
    return str(model.prompt(prompt))   

ementas_test['label_pred'] = ementas_test['text'].progress_apply(lambda x: classificar_pl(llama,x))

100%|██████████| 10/10 [00:52<00:00,  5.22s/it]


In [6]:
ementas_test.head(2)

,id,text,label_tema,label_desfavoravel,proponente_mulher,partido,urlInteiroTeor,textoInteiroTeor,label_pred
41,PL 77/2021,"Altera a Lei nº 11.340, de 7 de agosto de 2006...",direitos sociais,0,M,PSL,https://www.camara.leg.br/proposicoesWeb/prop_...,CÂMARA DOS DEPUTADOS\nGabinete do Deputado Ale...,0
1,PDL 92/2023,"Susta os efeitos da Portaria nº 665, de 23 dem...",genero,1,M,REPUBLICANOS,https://www.camara.leg.br/proposicoesWeb/prop_...,CÂMARA DOS DEPUTADOS\nGabinete do Deputado Mes...,1


In [7]:
from sklearn.metrics import classification_report
print(classification_report(ementas_test['label_desfavoravel'].astype(int), ementas_test['label_pred'].astype(int)))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88         8
           1       0.50      0.50      0.50         2

    accuracy                           0.80        10
   macro avg       0.69      0.69      0.69        10
weighted avg       0.80      0.80      0.80        10



# Deepseek


In [8]:
deepseek = llm.get_model("deepseek-r1")
resposta = classificar_pl(deepseek, "Define que a Lei Maria da Penha só é aplicável para mulheres de baixa renda")

resposta

'<think>\nHmm, preciso analisar se este projeto de lei (PL) é favorável ou contrário ao direito das mulheres. O texto diz: "Define que a Lei Maria da Penha só é aplicável para mulheres de baixa renda". A Lei Maria da Penha foi criada no Brasil para combater a violência doméstica contra mulheres, protegendo-as com medidas como multas e prisão em caso de agressões.\n\nSe um PL limitar essa lei a apenas mulheres de baixa renda, isso seria discriminatório. O direito das mulheres deve ser universal, cobrindo todas as classes sociais sem exceções baseadas na condição econômica. Isso parece violar o princípio da igualdade e da não discriminação.\n\nA resposta precisa ser apenas 1 ou 0: "1" significa contrário ao direito das mulheres, "0" significa favorável. Nesse caso, limitar a aplicação da lei por renda é claramente prejudicial, pois exclui mulheres de outras classes que também podem sofrer violência.\n\nPortanto, o PL não está alinhado com os valores do direito universal das mulheres e de

In [9]:
def extrair_categoria(texto):
    separador = "</think>"
    resposta  = texto.split(separador)[1].strip()
    return resposta

extrair_categoria(resposta)

'1'

In [10]:
# %%time
# def classificar_pl(model, texto):
#     prompt = f"""
#     Classifique se o PL é favorável ou contrário ao direito das mulheres. Responda apenas com 1 ou 0, sem explicações, de acordo com as definições:

#     1: significa que o PL é desfavorável ao direito das mulheres.
#     0: significa que o PL é favorável ao direito das mulheres.  

#     ---

#     Texto: {texto}

#     Classes: 1, 0
#     """
    
#     resposta = str(model.prompt(prompt))
    
#     separador = "</think>"
#     resposta  = resposta.split(separador)[1].strip()
    
#     return resposta

classificar_pl(deepseek, "Define que a Lei Maria da Penha só é aplicável para mulheres de baixa renda")

'<think>\nHmm, preciso analisar se o texto classificado está relacionado com algo favorável ou desfavorável ao direito das mulheres. O usuário pediu apenas uma resposta binária (1 ou 0), sem explicações.\n\nO texto diz que a Lei Maria da Penha só seria aplicável para mulheres de baixa renda, caso isso acontecesse. Lembro que essa lei foi criada especificamente para combater violência doméstica contra mulheres, garantindo medidas como prisão temporária e responsabilização do agressor.\n\nSe uma proposta limitasse a Lei Maria da Penha apenas a mulheres de baixa renda, isso seria claramente prejudicial. Isoladamente, não teria direito igual para todos os cidadãos, afetando especialmente quem já sofre mais discriminação.\n\nO usuário parece estar testando meu entendimento sobre classificação binária de textos legais. Ele quer uma resposta rápida e objetiva sem rodeios. Talvez esteja automatizando análises com base em regras simples que eu codifiquei mentalmente anteriormente.\n\nVou precis

In [ ]:
classificar_pl(deepseek, "Define que a Lei Maria da Penha só é aplicável para mulheres de baixa renda")

In [11]:

ementas_test['label_pred'] = ementas_test['text'].progress_apply(lambda x: classificar_pl(deepseek,x))

100%|██████████| 10/10 [14:29<00:00, 86.94s/it] 


In [12]:
separador = "</think>"
# separa a resposta
ementas_test["label_pred"] = ementas_test.label_pred.str.split(separador).str[1].str.strip()
# se não for nem 0 nem 1, vamos definir como 0
ementas_test["label_pred"] = ementas_test["label_pred"].apply(lambda x: 0 if x not in [0, 1] else x)

In [13]:

print(classification_report(ementas_test['label_desfavoravel'].astype(int), ementas_test['label_pred'].astype(int)))



              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       0.00      0.00      0.00         2

    accuracy                           0.80        10
   macro avg       0.40      0.50      0.44        10
weighted avg       0.64      0.80      0.71        10



/home/abitporu/Documents/abraji/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/abitporu/Documents/abraji/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/abitporu/Documents/abraji/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"